In [ ]:
import statsmodels.api as sm
from sklearn import datasets ## imports datasets from scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [ ]:
data = datasets.load_boston() ## loads Boston dataset from datasets library 

df = pd.DataFrame(data.data, columns=data.feature_names)
target = pd.DataFrame(data.target, columns=["MEDV"])
# Concatenate y in the dataframe
df_target = pd.concat([df,target], axis=1)

In [ ]:
df.columns

In [ ]:
df_target.columns

In [ ]:
import seaborn as sns
sns.pairplot(df_target[['MEDV','CHAS','LSTAT','CRIM','RM','AGE']])
plt.show()

# Compare Nested Models

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

In [ ]:
model_1 = smf.ols(formula='MEDV ~ CHAS + np.log(LSTAT)', data=df_target).fit()
model_2 = smf.ols(formula='MEDV ~ CHAS + np.log(LSTAT) + RM + AGE', data=df_target).fit()

In [ ]:
anovaResults = anova_lm(model_1, model_2)
print(anovaResults) 
# Notes:
# 1. You can ignore the warnings. The warnings are because the values in the first line are empty (NaN). 
#    numpy package will issue warnings for NaN values
# 2. ssr in the output are RSS or SSE

# Best Subset Selection

In [ ]:
# First create some dummy variables for categorical variables. This is another way of including categorical varaibles
df_dummy = pd.get_dummies(df, columns = ['CHAS'],drop_first = True) # Change categorical to one-hot
df_dummy.head(3)

In [ ]:
import itertools
#Importing tqdm for the progress bar
from tqdm import tnrange, tqdm_notebook

#Initialization variables
y = target
X = df_dummy.drop(columns=['ZN','INDUS','NOX','RAD']) # we don't include these variables in the model
k = 9
RSS_list, R_squared_list, adj_R_squared_list, AIC_list, BIC_list, feature_list = [],[],[],[],[],[]
numb_features = []

#Looping over k = 1 to k = 11 features in X
for k in tnrange(1,len(X.columns) + 1, desc = 'Loop...'): # note that for python range(2) = 0,1

    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(X.columns,k):
        X_c = sm.add_constant(X[list(combo)])       # we need to add constant term using sm.OLS
        model = sm.OLS(y, X_c).fit()                # run the regression model
        
        RSS_list.append(model.ssr)                  # model.ssr is the sum of squared residuals
        R_squared_list.append(model.rsquared)
        adj_R_squared_list.append(model.rsquared_adj)
        AIC_list.append(model.aic)
        BIC_list.append(model.bic)
        feature_list.append(combo)
        numb_features.append(len(combo))   

# Store the results in DataFrame
df_results = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list,'R_squared':R_squared_list,'features':feature_list,'adj_R_squared':adj_R_squared_list,'AIC':AIC_list,'BIC':BIC_list})

In [ ]:
df_results

In [ ]:
# Get the minimum RSS for each numb_features
df_minRSS = df_results[df_results.groupby('numb_features')['RSS'].transform(min) == df_results['RSS']] 
df_minRSS

In [ ]:
# Get max R^2 for each numb_features. 
# We can verify that it is the same as min RSS
df_maxRsqr = df_results[df_results.groupby('numb_features')['R_squared'].transform(max) == df_results['R_squared']] 
df_maxRsqr

In [ ]:
# Adding columns to the dataframe with RSS and R squared values of the best subset
# This is for plotting purpose only
df_results['min_RSS'] = df_results.groupby('numb_features')['RSS'].transform(min)
df_results['max_R_squared'] = df_results.groupby('numb_features')['R_squared'].transform(max)
df_results.head()

In [ ]:
fig = plt.figure(figsize = (16,6))
ax = fig.add_subplot(1, 2, 1)

ax.scatter(df_results.numb_features,df_results.RSS, alpha = .2, color = 'darkblue' )
ax.set_xlabel('# Features')
ax.set_ylabel('RSS')
ax.set_title('RSS - Best subset selection')
ax.plot(df_results.numb_features,df_results.min_RSS,color = 'r', label = 'Best subset')
ax.legend()

ax = fig.add_subplot(1, 2, 2)
ax.scatter(df_results.numb_features,df_results.R_squared, alpha = .2, color = 'darkblue' )
ax.plot(df_results.numb_features,df_results.max_R_squared,color = 'r', label = 'Best subset')
ax.set_xlabel('# Features')
ax.set_ylabel('R squared')
ax.set_title('R_squared - Best subset selection')
ax.legend()

plt.show()
# Ignore the warnings

In [ ]:
standards = ['AIC','BIC','adj_R_squared']
df_maxRsqr.index = df_maxRsqr.numb_features       # need to reset index to 1,2,3.. for this to plot the red X
fig = plt.figure(figsize = (18,6))

for i,v in enumerate(standards):
    ax = fig.add_subplot(1, len(standards), i+1)
    ax.plot(df_maxRsqr['numb_features'],df_maxRsqr[v], color = 'lightblue')
    ax.scatter(df_maxRsqr['numb_features'],df_maxRsqr[v], color = 'darkblue')
    if v == 'adj_R_squared':
        ax.plot(df_maxRsqr[v].idxmax(),df_maxRsqr[v].max(), marker = 'x', markersize = 20, color='r')
    else:
        ax.plot(df_maxRsqr[v].idxmin(),df_maxRsqr[v].min(), marker = 'x', markersize = 20, color='r')
    ax.set_xlabel('Number of predictors')
    ax.set_ylabel(v)

fig.suptitle('Subset selection using AIC, BIC, Adjusted R2', fontsize = 16)
plt.show()

# Forward Stepwise Selection

In [ ]:
#Initialization variables
y = target
X = df_dummy.drop(columns=['ZN','INDUS','NOX','RAD'])     # we don't include these variables in the model
k = 9

remaining_features = list(X.columns.values)               # Initialize the remaining features as all features
features = []
RSS_list, R_squared_list, adj_R_squared_list, AIC_list, BIC_list = [],[],[],[],[] 
features_list = dict()                                    # Intialize feature list using dictionary. This is one way

for i in range(1,k+1):
    best_RSS = np.inf                                     # initialize the best_RSS in each round to be infinity
    
    for combo in itertools.combinations(remaining_features,1): # iterate through all remaining features
        
        X_c = sm.add_constant(X[list(combo) + features])  # we need to add constant term using sm.OLS
        model = sm.OLS(y, X_c).fit()

        if model.ssr < best_RSS:                          # compare the RSS value with the smallest value in this round
            best_RSS = model.ssr                          # update the best value
            best_R_squared = model.rsquared               # update best best_R_squared
            best_feature = combo[0]                       # the best feature in this round
            best_aic = model.aic
            best_bic = model.bic
            best_adj_R_squared = model.rsquared_adj

    #Updating variables for next loop
    features.append(best_feature)                         # add the best feature in the features set
    remaining_features.remove(best_feature)               # remove it from candidate set
    
    #Saving values for plotting
    RSS_list.append(best_RSS)
    R_squared_list.append(best_R_squared)
    AIC_list.append(best_aic)
    BIC_list.append(best_bic)
    adj_R_squared_list.append(best_adj_R_squared)
    features_list[i] = features.copy()

In [ ]:
# store results in df_results, which is a joint of df_features and df_values
df_features = pd.DataFrame({'features':features_list})
df_values = pd.DataFrame({'RSS':RSS_list, 'R_squared': R_squared_list,'AIC':AIC_list,'BIC':BIC_list, 'adj_R_squared': adj_R_squared_list})
df_values.index += 1  # shift the index by 1 to get aligned with df_features
df_results = pd.concat([df_features,df_values], axis=1, join='inner')
df_results['numb_features'] = df_results.index
df_results

In [ ]:
standards = ['AIC','BIC','adj_R_squared']
fig = plt.figure(figsize = (18,6))

for i,v in enumerate(standards):
    ax = fig.add_subplot(1, len(standards), i+1)
    ax.plot(df_results['numb_features'],df_results[v], color = 'lightblue')
    ax.scatter(df_results['numb_features'],df_results[v], color = 'darkblue')
    if v == 'adj_R_squared':
        ax.plot(df_results[v].idxmax(),df_results[v].max(), marker = 'x', markersize = 20, color='r')
    else:
        ax.plot(df_results[v].idxmin(),df_results[v].min(), marker = 'x', markersize = 20, color='r')
    ax.set_xlabel('Number of predictors')
    ax.set_ylabel(v)

fig.suptitle('Subset selection using AIC, BIC, Adjusted R2', fontsize = 16)
plt.show()

## We can define a function to ensulate the two methods

In [ ]:
def best_subset(y, X):
    import itertools
    from tqdm import tnrange, tqdm_notebook #Importing tqdm for the progress bar
    RSS_list, R_squared_list,adj_R_squared_list, AIC_list, BIC_list, feature_list = [],[],[],[],[],[]
    numb_features = []
    k = len(X.columns)
    #Looping over k = 1 to k = 11 features in X
    for k in tnrange(1,len(X.columns) + 1, desc = 'Loop...'): # note that for python range(2) = 0,1

        #Looping over all possible combinations: from 11 choose k
        for combo in itertools.combinations(X.columns,k):
            X_c = sm.add_constant(X[list(combo)])       # we need to add constant term using sm.OLS
            model = sm.OLS(y, X_c).fit()                # run the regression model
            RSS_list.append(model.ssr)                  # model.ssr is the sum of squared residuals
            R_squared_list.append(model.rsquared)
            adj_R_squared_list.append(model.rsquared_adj)
            AIC_list.append(model.aic)
            BIC_list.append(model.bic)
            feature_list.append(combo)
            numb_features.append(len(combo))   

    # Store the results in DataFrame
    df_results = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list,'R_squared':R_squared_list,
                               'features':feature_list,'adj_R_squared':adj_R_squared_list,'AIC':AIC_list,'BIC':BIC_list})
    return df_results

In [ ]:
def forward_stepwise(y, X, remaining_features):
    features = []
    RSS_list, R_squared_list, adj_R_squared_list, AIC_list, BIC_list = [],[],[],[],[] 
    features_list = dict()                                    # Intialize feature list using dictionary. This is one way
    k = len(remaining_features)
    for i in range(1,k+1):
        best_RSS = np.inf                                     # initialize the best_RSS in each round to be infinity

        for combo in itertools.combinations(remaining_features,1): # iterate through all remaining features

            X_c = sm.add_constant(X[list(combo) + features])  # we need to add constant term using sm.OLS
            model = sm.OLS(y, X_c).fit()

            if model.ssr < best_RSS:                          # compare the RSS value with the smallest value in this round
                best_RSS = model.ssr                          # update the best value
                best_R_squared = model.rsquared               # update best best_R_squared
                best_feature = combo[0]                       # the best feature in this round
                best_aic = model.aic
                best_bic = model.bic
                best_adj_R_squared = model.rsquared_adj

        #Updating variables for next loop
        features.append(best_feature)                         # add the best feature in the features set
        remaining_features.remove(best_feature)               # remove it from candidate set

        #Saving values for plotting
        RSS_list.append(best_RSS)
        R_squared_list.append(best_R_squared)
        AIC_list.append(best_aic)
        BIC_list.append(best_bic)
        adj_R_squared_list.append(best_adj_R_squared)
        features_list[i] = features.copy()

    # store results in df_results, which is a joint of df_features and df_values
    df_features = pd.DataFrame({'features':features_list})
    df_values = pd.DataFrame({'RSS':RSS_list, 'R_squared': R_squared_list,'AIC':AIC_list,'BIC':BIC_list, 'adj_R_squared': adj_R_squared_list})
    df_values.index += 1  # shift the index by 1 to get aligned with df_features
    df_results = pd.concat([df_features,df_values], axis=1, join='inner')
    df_results['numb_features'] = df_results.index
    return df_results

In [ ]:
def plot_selection(df_results, standards):
    fig = plt.figure(figsize = (18,6))

    for i,v in enumerate(standards):
        ax = fig.add_subplot(1, len(standards), i+1)
        ax.plot(df_results['numb_features'],df_results[v], color = 'lightblue')
        ax.scatter(df_results['numb_features'],df_results[v], color = 'darkblue')
        if v == 'adj_R_squared':
            ax.plot(df_results[v].idxmax(),df_results[v].max(), marker = 'x', markersize = 20, color='r')
        else:
            ax.plot(df_results[v].idxmin(),df_results[v].min(), marker = 'x', markersize = 20, color='r')
        ax.set_xlabel('Number of predictors')
        ax.set_ylabel(v)

    fig.suptitle('Subset selection using ' + ", ".join(standards), fontsize = 16)
    plt.show()

## Best subset vs. forward stepwise

In [ ]:
#Initialization variables
y = target
X = df_dummy.drop(columns=['ZN','INDUS','NOX','RAD','AGE','PTRATIO','B']) # we don't include these variables in the model
df_results_BS = best_subset(y, X)

In [ ]:
#Initialization variables
y = target
X = df_dummy.drop(columns=['ZN','INDUS','NOX','RAD','AGE','PTRATIO','B'])     # we don't include these variables in the model
remaining_features = list(X.columns.values)               # Initialize the remaining features as all features
df_results_FS = forward_stepwise(y, X, remaining_features)

In [ ]:
df_results_BS

In [ ]:
df_results_FS

In [ ]:
df_maxRsqr = df_results_BS[df_results_BS.groupby('numb_features')['R_squared'].transform(max) == df_results_BS['R_squared']] # Get max R^2 for each number of predictors
df_maxRsqr.index = df_maxRsqr.numb_features
standards = ['AIC','BIC','adj_R_squared']
plot_selection(df_maxRsqr, standards)

In [ ]:
standards = ['AIC','BIC','adj_R_squared']
plot_selection(df_results_FS, standards)